In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_12492\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
base_url = "https://www.justwatch.com"
link = "https://www.justwatch.com/us/movies?genres=act&release_year_from=2022"

In [196]:
genres = ['cmy', 'act', 'doc', 'fnt', 'hrr', 'msc', 'rms', 'spt', 'wsn', 'ani', 'crm', 'drm', 'hst', 'fml', 'trl', 'scf', 'war', 'rly']
years = range(1900, 202, 1)

for i in genres:
    for j in years:
        link = f"https://www.justwatch.com/us/movies?genres={i}&release_year_from={j}"
        
        print(link)

In [5]:
url = link
driver.get(url)
ScrollNumber = 50
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollTo(1,50000)")
    WebDriverWait(driver, 10)
    time.sleep(5)
page_source =driver.page_source

In [6]:
titleLinks = []
soup=BeautifulSoup(page_source,'html.parser')
productlist = soup.find("div",{"listlayout":"Grid"})
titles = productlist.find_all("div", {"class":"title-list-grid__item"})
for title in titles:
    title_link = title.find("a").get('href')
    title_link = base_url+title_link
    titleLinks.append(title_link)

In [7]:
len(titleLinks)

648

In [87]:
labels  = []
values = []
url = "https://www.justwatch.com/us/movie/uncharted-2017"
def get_detail(soup_object):
    table_info = soup_object.find("div", {"class":"title-info"})
    rows = table_info.find_all("div", {"class":"detail-infos"})
    for row in rows:
        label = row.find("div", {"class":"detail-infos__subheading"}).text.replace('\n',"")
        value = row.find("div", {"class":"detail-infos__value"}).text.replace('\n',"")
        labels.append(label)
        values.append(value)
        
    detail = {labels[i]: values[i] for i in range(len(labels))}
        
    return detail

In [89]:
def get_cast(soup_object):
    cast = []
    actors = soup_object.find_all("div", {"class":"title-credits__actor"})
    for actor in actors:
        actor_name = actor.text.replace('\n',"")
        cast.append(actor_name)
    return cast

In [166]:
class Pricing:
    def __init__(self, soup_object):
        self.soup_object = soup_object

    def get_stream_options(self):
        stream_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)


        return stream_list


    def get_rent_options(self):
        rent_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list

    def get_buy_options(self):
        buy_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list


In [187]:
def get_synopsis(soup_object):
    driver.get(soup_object)       
    synopsis = driver.find_element(By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/p').text
    return synopsis

get_rent_options(soup)

In [191]:
from pydoc import synopsis


data= []
for title in titleLinks:
    from datetime import datetime
    f = requests.get(title,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')
    prices = Pricing(hun)
    
    try:
        detail = get_detail(hun)
    except:
        detail = None
        
    try:
        title_name = hun.find("div", {"class":"title-block"}).text.replace('\n',"")
    except:
        title_name = None
    
    try:
        streaming_options = prices.get_stream_options()
    except:
        streaming_options = None
        
    try:
        renting_options = prices.get_rent_options()
    except:
        renting_options = None
    
    try:
        synopsis = get_synopsis(title)
    except:
        synopsis = None
    
    try:
        buying_options = prices.get_buy_options()
    except:
        buying_options = None
        
    try:
        cast = get_cast(hun)
    except:
        cast = None
    
    title_url = title
    
    
    
      
    tv = {"title":title_name, "detail":detail, "synopsis":synopsis, "streaming on":streaming_options, "buy from":buying_options, "rent on":renting_options,"cast":cast,"tite url": title_url}
    
    data.append(tv)

In [194]:
import pandas as pd
df = pd.DataFrame.from_dict(data)
df

,title,detail,synopsis,streaming on,buy from,rent on,cast,tite url
0,Prey (2022),"{'Rating': ' 89% 7.2 (132k) ', 'Genres': 'Mys...",Set more than 300 years ago and following the ...,[{'Hulu': 'Subs 4K '}],[],[],"[ Amber Midthunder Naru, Dakota Beavers Taabe...",https://www.justwatch.com/us/movie/prey-2022
1,Everything Everywhere All at Once (2022),"{'Rating': ' 92% 8.2 (186k) ', 'Genres': 'Act...",The fate of the world depends on a Chinese imm...,[],"[{'Spectrum On Demand': '$6.99 HD '}, {'Apple ...","[{'Redbox': '$3.99 '}, {'Amazon Video': '$4.99...","[ Michelle Yeoh Evelyn Wang, Stephanie Hsu Jo...",https://www.justwatch.com/us/movie/everything-...
2,Top Gun: Maverick (2022),"{'Rating': ' 97% 8.5 (315k) ', 'Genres': 'Dra...","After three decades of service, Pete “Maverick...",[],"[{'Apple iTunes': '$19.99 4K '}, {'Amazon Vide...",[],"[ Tom Cruise Capt. Pete 'Maverick' Mitchell, ...",https://www.justwatch.com/us/movie/top-gun-mav...
3,Jurassic World Dominion (2022),"{'Rating': ' 76% 5.7 (122k) ', 'Genres': 'Sci...","In the final part of the trilogy, Jurassic Wor...","[{'Peacock': 'Ads HD '}, {'Peacock Premium': '...","[{'Apple iTunes': '$19.99 4K '}, {'Amazon Vide...",[],"[ Chris Pratt Owen Grady, Bryce Dallas Howard...",https://www.justwatch.com/us/movie/jurassic-wo...
4,Uncharted (2022),"{'Rating': ' 78% 6.4 (175k) ', 'Genres': 'Act...",Nathan Drake is recruited by a veteran treasur...,[{'Netflix': 'Subs 4K '}],"[{'Spectrum On Demand': '$6.99 HD '}, {'Apple ...","[{'Apple iTunes': '$5.99 4K '}, {'Amazon Video...","[ Tom Holland Nathan Drake, Mark Wahlberg Vic...",https://www.justwatch.com/us/movie/uncharted-2017
...,...,...,...,...,...,...,...,...
643,Detective Inspector (2022),"{'Rating': ' 1% ', 'Genres': 'Action & Adventu...",None,[],[],[],[],https://www.justwatch.com/us/movie/detective-i...
644,Una preghiera per Giuda (2022),"{'Rating': ' 1% ', 'Genres': 'Action & Adventu...",None,[],[],[],[],https://www.justwatch.com/us/movie/una-preghie...
645,The Mighty Six Movie 2: Rise of the Villains ...,"{'Rating': ' 1% ', 'Genres': 'Action & Adventu...",None,[],[],[],[],https://www.justwatch.com/us/movie/the-mighty-...
646,Ace (2022),"{'Rating': ' 1% ', 'Genres': 'Animation , Acti...",None,None,None,None,[],https://www.justwatch.com/us/movie/ace-2022


In [197]:
df.to_csv("2022_action.csv")